## 해당 장에서는 전이학습 및 Pytorch를 활용해서 딥러닝을 돌리는 방법에 대해서 배우는 시간을 가졌다.

In [1]:
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 1. 데이터 폴더 경로 정하기(Making Folder and Directory)

In [3]:
import os
import urllib.request
import zipfile

In [4]:
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [5]:
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = os.path.join(data_dir, "imagenet_class_index.json")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [6]:
url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  
    zip.close()  

    os.remove(save_path)

## 2. 이미지를 변환하는 클래스를 만든다.

In [7]:
class ImageTransform():
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            "train" :  transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale = (0.5, 1.0)
                ),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            "val" : transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
    def __call__(self, img, phase = "train"):
        return self.data_transform[phase](img)

__call__ 메서드는 객체를 호출 가능하게 만든다. ImageTransform 객체가 이미지와 선택적인 인자와 함께 호출될 때, self.data_transform[phase](img)가 리턴된다.

In [14]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

## 3. Train과 Val 데이터 셋의 경로를 지정한다

In [15]:
def make_datapath_list(phase = "train"):
    
    rootpath = "./data/hymenoptera_data/"
    target_path = osp.join(rootpath+phase+'/**/*.jpg')
    print(target_path)

    path_list = []  

    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list

In [16]:
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [17]:
class HymenopteraDataset(data.Dataset):
    
    def __init__(self, file_list, transform = None, phase = "train"):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.file_list)
    
    # __getitem__을 작동시킬려고 하려면 dataset(index)
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)

        img_transformed = self.transform(img, self.phase)

        if self.phase == "train":
            label = img_path[30:34]
        elif self.phase == "val":
            label = img_path[28:32]

        if label == "ants":
            label = 0
        elif label == "bees":
            label = 1
        
        return img_transformed, label

        
        

In [18]:
train_dataset = HymenopteraDataset(
    file_list = train_list, transform = ImageTransform(size, mean, std), phase = 'train') 

val_dataset = HymenopteraDataset(
    file_list = val_list, transform = ImageTransform(size, mean, std), phase = 'val')


index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
1


## 4. DataLoader를 통해 데이터 셋을 텐서 형태로 변환해주는 작업을 거친다.

In [19]:
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)


dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

batch_iterator = iter(dataloaders_dict["train"])  
inputs, labels = next(
    batch_iterator)  
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0])


## 5. 전이학습 할 모델 설정

In [ ]:
# 기존에 사용했던 전이 학습층을 사용해서 학습을 진행함
use_pretrained = True
net = models.vgg16(pretrained = use_pretrained)

# 마지막 fc-layer의 input을 4096, output을 2개로 나올 수 있도록 함(vgg-net의 6번째 층)
net.classifier[6] = nn.Linear(in_features = 4096, out_features = 2)

net.train()

## 6. 목적적함수 정의:

트레인을 하기 전에 목적함수(Loss Function)을 정의한다.

In [ ]:
criterion = nn.CrossEntropyLoss()

## 7. 전이학습을 시킬 net 모델(VGGNET)의 일부 파라미터를 업데이트 하는 과정을 거친다.

In [ ]:
params_to_update = []

update_param_names = ['classifier.6.weight', 'classifier.6.bias']

for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

## 8. 업데이트한 파라미터를 반영하여 옵티마이저를 설정한다.

In [ ]:
optimizer = optim.SGD(params = params_to_update, lr = 0.001, momentum = 0.9)

In [ ]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and (phase == "train"):
                continue

            for inputs, labels in tqdm(dataloaders_dict[phase]):# 미니 배치를 가져오고 있다.

                optimizer.zero_grad()# 옵티마이저의 그라디언트를 초기화 한다

                with torch.set_grad_enabled(phase == "train"):# 학습 단계에서만 그라디언트를 계산하도록 한다. 검증 시에는 경사를 계산할 필요가 없다. 파이토치에서 학습과 검증으로 네트워크 모드를 전환하는 것은 드롭아웃 층과 같이 학습 및 검증에 동작이 서로 다른 층이 있기 때문이다.
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()#학습 단계에서 역전파를 수행하고
                        optimizer.step()#옵티마이저를 사용하여 가중치를 업데이트한다.

                    epoch_loss += loss.item() * inputs.size(0) # 에폭 손실을 업데이트 한다.
                    # epoch_loss는 현재까지의 누적 손실 값을 나타내며, 각 미니배치의 손실 값을 미니배치의 크기로 가중하여 더해줍니다. 이는 각 미니배치의 손실을 모두 더한 후에 전체 데이터셋의 크기로 나누어 평균 손실 값을 계산하는데 사용됩니다.

                    epoch_corrects += torch.sum(preds == labels.data) #정확하게 예측된 수를 업데이트 한다
            
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
        

옵티마이저의 그라디언트를 초기화 하는 이유는 새로운 미니배치를 사용하여 새로운 경사하강 단계를 시작하기 전에 이전 단계의 그라디언트가 현재 단계에 영향을 미치지 않도록 하기 위해서이다. (요약) 미니 배치 간에 그라디언트 영향을 주지 않기 때문이다. 